In [ ]:
import pandas as pd

file_name=['01','02','03','04','06','08','14','39']
data=[]

for i in range(len(file_name)):
    data.append(pd.read_csv('../data/'+file_name[i]+'_tracks.csv'))


print(data[2].head())
